In [1]:
import sys
import os

# Add the src directory to Python path so model.py can find ssn and net modules
sys.path.append(os.path.abspath('../src'))

from src.model import model
from src.PDAP import retrain

import numpy as np
from loguru import logger
import torch

In [2]:
# load the data
path = '../data_result/raw_data/gauss_cos_31x31.npy'# Initialize the weights
data = np.load(path)
logger.info(f"Loaded data with shape: {data.shape}, dtype: {data.dtype}")

2025-09-21 19:15:50.905 | INFO     | __main__:<module>:4 - Loaded data with shape: (961,), dtype: [('x', '<f8', (2,)), ('dv', '<f8', (2,)), ('v', '<f8')]


In [3]:
# Initialize the parameter
power = 2.1
M = 50 # number greedy insertion selected
num_iterations = 10
loss_weights = (1.0, 0.0)
pruning_threshold = 1e-15

gamma = 5.0
alpha = 1e-5
lr_adam = 1e-5
regularization = (gamma, alpha) 
th = 0.0


In [4]:
# Initialize the models
model_1 = model(activation=torch.relu, power=power, regularization=regularization, optimizer='Adam', loss_weights=loss_weights, th = th)
model_2 = model(activation=torch.relu, power=power, regularization=regularization, optimizer='SSN_TR', loss_weights=loss_weights, th=th, train_outerweights=True)

2025-09-21 19:15:53 | INFO     | src.model:_configure_logger:105 - Model initialized


2025-09-21 19:15:53 | INFO     | src.model:_configure_logger:105 - Model initialized


In [5]:
# prepare the data
data_train, data_valid = model_1._prepare_data(data)
# Set up the initializing weights and bias


2025-09-21 19:15:54 | INFO     | src.model:_prepare_data:150 - Training set: 864 samples, Validation set: 97 samples
2025-09-21 19:15:54 | INFO     | src.model:_prepare_data:153 - Data ranges - x: [-1.00, 1.00], v: [-0.72, 1.00], dv: [-5.86, 5.86]


In [6]:
best_iteration, history = retrain(data_train, data_valid, model_1, model_2, num_iterations, M, pruning_threshold)

2025-09-21 19:15:56 | INFO     | src.PDAP:retrain:60 - Iteration 0 - Starting...
2025-09-21 19:15:56 | INFO     | src.model:train:307 - Starting network training session
2025-09-21 19:15:56 | INFO     | src.model:_create_network:194 - Creating network with 50 neurons
2025-09-21 19:15:58 | INFO     | src.model:_setup_optimizer:244 - Using Adam optimizer with lr=0.01
2025-09-21 19:15:58 | INFO     | src.model:train:322 - Training hyperparameters: iterations=5000, batch_size=1620, display_every=1000
2025-09-21 19:15:58 | INFO     | src.model:train:323 - Loss weights: value=1.0, gradient=0.0
2025-09-21 19:15:59 | INFO     | src.model:train:362 - Epoch 0: Train Loss = 0.748650, Val Loss = 0.505255
2025-09-21 19:16:05 | INFO     | src.model:train:362 - Epoch 1000: Train Loss = 0.060254, Val Loss = 1.833949
2025-09-21 19:16:11 | INFO     | src.model:train:362 - Epoch 2000: Train Loss = 0.049605, Val Loss = 1.991746
2025-09-21 19:16:16 | INFO     | src.model:train:362 - Epoch 3000: Train Loss 

KeyboardInterrupt: 

In [ ]:
# def save(self):
#     """Save training history."""
#     timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
#     filename = f"training_history_{timestamp}.pkl"
#     filepath = os.path.join(self.stats_dir, filename)
    
#     with open(filepath, 'wb') as f:
#         pickle.dump(self.history, f)
        
#     logger.info(f"Saved training history to {filepath}")
#     return filepath

2025-09-21 17:31:47 | INFO     | src.model:_configure_logger:104 - Model initialized


In [9]:
model_1.train(
    data_train=data_train,
    data_valid=data_valid,
    inner_weights=init_weights, 
    inner_bias=init_bias,
    iterations=1000,
    display_every=200,
)

2025-09-21 17:32:28 | INFO     | src.model:train:306 - Starting network training session
2025-09-21 17:32:28 | INFO     | src.model:_create_network:193 - Creating network with 50 neurons
2025-09-21 17:32:28 | INFO     | src.model:_setup_optimizer:243 - Using Adam optimizer with lr=0.01
2025-09-21 17:32:29 | INFO     | src.model:train:321 - Training hyperparameters: iterations=1000, batch_size=1620, display_every=200
2025-09-21 17:32:29 | INFO     | src.model:train:322 - Loss weights: value=1.0, gradient=0.0
2025-09-21 17:32:29 | INFO     | src.model:train:361 - Epoch 0: Train Loss = 0.758605, Val Loss = 1.770856
2025-09-21 17:32:30 | INFO     | src.model:train:361 - Epoch 200: Train Loss = 0.138397, Val Loss = 0.556594
2025-09-21 17:32:31 | INFO     | src.model:train:361 - Epoch 400: Train Loss = 0.107812, Val Loss = 1.682160
2025-09-21 17:32:32 | INFO     | src.model:train:361 - Epoch 600: Train Loss = 0.098375, Val Loss = 1.937616
2025-09-21 17:32:33 | INFO     | src.model:train:361 

{'loss_history': [0.7586048966332555,
  0.13839654131324466,
  0.10781152371223449,
  0.09837455612629147,
  0.09292016148593406],
 'val_history': [1.7708560374591833,
  0.5565938616000192,
  1.6821601476086416,
  1.9376158157305623,
  1.8959011655118543],
 'best_epoch': 200,
 'config': {'optimizer': 'Adam',
  'activation': 'relu',
  'power': 2.1,
  'regularization': {'gamma': 5.0, 'alpha': 1e-05, 'th': 0.5},
  'loss_weights': {'value': 1.0, 'grad': 0.0},
  'training_percentage': 0.9,
  'train_outerweights': False,
  'iterations': 1000,
  'display_every': 200}}

In [ ]:
# Training with improved logging
KP_logger_L2_nc, weight_raw, bias_raw, outerweight_raw = retrain(
    data_train, data_valid, 
    model_1, model_2, model_result, weight_raw, bias_raw, outerweight_raw,
    num_iterations, M, alpha, pruning_threshold, power, gamma
)

logger.info("Training completed with improved logging")

Reached 6.04e-3 at 179 neurones model is about 126 neurons

In [ ]:
# =============================================================================
# PLOT: Weight space visualization in polar coordinates
# Shows the distribution of weights in 2D space for the BEST model (lowest validation loss)
# =============================================================================

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Extract weights from the current training run
weights_run = KP_logger_L2_nc.history['weights']
biases_run = KP_logger_L2_nc.history['biases']
neurons_run = KP_logger_L2_nc.history['neuron_count']

print(f"Training run: {len(weights_run)} iterations, max neurons: {max(neurons_run)}")

# Use the BEST model (lowest validation loss) instead of the iteration with most neurons
print("Using BEST model (lowest validation loss)")
weights_optimal = KP_logger_L2_nc.history['best_weights']
b_optimal = KP_logger_L2_nc.history['best_biases'].reshape(1, -1)   # (1, n)
best_loss = KP_logger_L2_nc.history['best_test_loss']
print(f"Best validation loss: {best_loss:.6f}")

a_optimal = weights_optimal.T                         # (2, n)
Z = a_optimal / (1 + b_optimal) 

# Create polar coordinate visualization
fig, ax = plt.subplots(1, 1, figsize=(10, 8), subplot_kw={'projection': 'polar'})

# Compute angles and radii in weight space (2D)
angles = np.arctan2(a_optimal[1], a_optimal[0])
r_sphere = np.sqrt(np.sum(a_optimal**2, axis=0) + (b_optimal.flatten())**2)

# Plot in polar coordinates
radius = np.ones_like(angles)
ax.scatter(angles, radius, c=b_optimal.flatten(), cmap='coolwarm', alpha=0.85, s=60)
ax.set_title(f'Weight Space - BEST Model\nNeurons: {weights_optimal.shape[0]}', fontsize=14)
ax.grid(True, alpha=0.5)

# Save the figure
plt.savefig('../data_result/plot/weights_polar_analysis_best.png', dpi=300, bbox_inches='tight')
print(f"Polar coordinate analysis saved to ../data_result/plot/weights_polar_analysis_best.png")

# Show plot
plt.show()

## Test with the L1 Penalty ##

In [ ]:
# Initialize the parameter
power = 2.1
M = 50 # number greedy insertion selected
num_iterations = 10
loss_weights = (1.0, 0.0)
pruning_threshold = 1e-13

gamma = 1e-10
alpha = 1e-5
regularization = (gamma, alpha) 
th = 1.0

The comparable model is about: 207 neurons

In [ ]:
# Initialize the model 
model_1 = model(torch.relu, power, regularization, optimizer='Adam', loss_weights = loss_weights)
model_2 = model_outerweights(torch.relu, power, regularization, optimizer='SSN_TR', loss_weights = loss_weights, th = th)

# Prepare data
data_train, data_valid = model_1._prepare_data(0.8, data)

# Set up the initializing weights and bias
init_weights = np.random.randn(M, 2) * 0.1
init_bias = np.random.randn(M)

model_result, weight_raw, bias_raw, outerweight_raw = model_1.train(
    data_train=data_train,
    data_valid=data_valid,
    inner_weights=init_weights, 
    inner_bias=init_bias,
    iterations=1000,
    display_every=200,
)
logger.info("Initialization done"); logger.info(f"Initial weights shape: {weight_raw.shape}, bias shape: {bias_raw.shape}")

# Training with improved logging
KP_logger_L2_lasso, weight_raw_1, bias_raw_1, outerweight_raw_1 = retrain(
    data_train, data_valid, model_1, model_2, model_result, weight_raw, bias_raw, outerweight_raw,
    num_iterations, M, alpha, pruning_threshold, power, gamma
)

logger.info("Training completed with improved logging")


Reached 6e-3 at 157 neurones, 3.6e-3 at 231 neurons

In [ ]:
# =============================================================================
# PLOT: Weight space visualization in polar coordinates
# Shows the distribution of weights in 2D space for the BEST model (lowest validation loss)
# =============================================================================

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Extract weights from the current training run
weights_run = KP_logger_L2_lasso.history['weights']
biases_run = KP_logger_L2_lasso.history['biases']
neurons_run = KP_logger_L2_lasso.history['neuron_count']

print(f"Training run: {len(weights_run)} iterations, max neurons: {max(neurons_run)}")

# Use the BEST model (lowest validation loss) instead of the iteration with most neurons
if 'best_weights' in KP_logger_L2_lasso.history:
    print("Using BEST model (lowest validation loss)")
weights_optimal = KP_logger_L2_lasso.history['best_weights']
b_optimal = KP_logger_L2_lasso.history['best_biases'].reshape(1, -1)   # (1, n)
best_loss = KP_logger_L2_lasso.history['best_test_loss']
print(f"Best validation loss: {best_loss:.6f}")

a_optimal = weights_optimal.T                         # (2, n)
Z = a_optimal / (1 + b_optimal) 

# Create polar coordinate visualization
fig, ax = plt.subplots(1, 1, figsize=(10, 8), subplot_kw={'projection': 'polar'})

# Compute angles and radii in weight space (2D)
angles = np.arctan2(a_optimal[1], a_optimal[0])
r_sphere = np.sqrt(np.sum(a_optimal**2, axis=0) + (b_optimal.flatten())**2)

# Plot in polar coordinates
radius = np.ones_like(angles)
ax.scatter(angles, radius, c=b_optimal.flatten(), cmap='coolwarm', alpha=0.85, s=60)
ax.set_title(f'Weight Space - BEST Model (L1 Penalty)\nNeurons: {weights_optimal.shape[0]}', fontsize=14)
ax.grid(True, alpha=0.5)

# Save the figure
plt.savefig('../data_result/plot/weights_polar_analysis_best_l1.png', dpi=300, bbox_inches='tight')
print(f"Polar coordinate analysis saved to ../data_result/plot/weights_polar_analysis_best_l1.png")

# Show plot
plt.show()


# Optional: Show weight evolution across iterations
if len(weights_run) > 1:
    print(f"\n=== Weight Evolution ===")
    print("Neuron counts across iterations:")
    for i, count in enumerate(neurons_run):
        print(f"Iteration {i}: {count} neurons")